In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nptdms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nptdms: filename=nptdms-1.10.0-py3-none-any.whl size=108456 sha256=6d63ce786d27a0517ca4046a20a776bdf9008ad08b73c3ae4b39da94bd6bd8ee
  Stored in directory: /root/.cache/pip/wheels/1b/4b/17/21e8b03b37ea51ce7ec9f5570cdf0decca93f537d61c06880f
Successfully built nptdms


In [4]:
import os
import numpy as np
import pandas as pd
from nptdms import TdmsFile
import pywt
from scipy.fft import rfft
from tensorflow.keras.models import load_model

import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib
from tqdm import tqdm  # 진행 상태 표시용 (선택적)

전처리 파이프라인

In [5]:
# --- 설정값 ---
data_dir = "Train Set"
# 진동 채널만 사용
channels_raw = ['CH1', 'CH2', 'CH3', 'CH4']
other_channels_raw = ['Torque[Nm]', 'TC SP Front[℃]', 'TC SP Rear[℃]']
all_target_channels = channels_raw + other_channels_raw # 모든 대상 채널

window_size = 25600
overlap = 0.5
wavelet = 'db4'
level = 3
top_k = 10 # WPT+FFT에서 상위 k개 주파수 특징
sampling_rate = 25600 # 진동 데이터 샘플링 레이트
step = int(window_size * (1 - overlap))


# --- 슬라이딩 윈도우 함수 ---
def sliding_window(data, window_size, overlap):
    step = int(window_size * (1 - overlap))
    return np.array([data[i:i+window_size] for i in range(0, len(data) - window_size + 1, step)])

# --- WPT+FFT 특징 추출 함수  ---
def extract_wpt_fft_features(signal, wavelet='db4', level=3, top_k=10):
    wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=level)
    nodes = [node.path for node in wp.get_level(level, 'freq')]
    features = []
    for node in nodes:
        coeffs = wp[node].data
        if len(coeffs) > 0: # coeffs가 비어있지 않은지 확인
            fft_vals = np.abs(rfft(coeffs))
            if len(fft_vals) >= top_k:
                top_features = np.sort(fft_vals)[-top_k:]
            else: # top_k보다 작으면 있는 그대로 사용하고 나머지는 0으로 채움
                top_features = np.pad(np.sort(fft_vals), (top_k - len(fft_vals), 0), 'constant', constant_values=0)
            features.extend(top_features)
        else: # coeffs가 비어있으면 0으로 채움
            features.extend([0.0] * top_k)
    return np.array(features)

# --- TDMS 파일 데이터 로드 함수 ---
def _load_tdms_data(file_path, channels_raw, other_channels_raw):
    try:
        tdms_file = TdmsFile.read(file_path)
        if len(tdms_file.groups()) < 2:
            print(f"[경고] {file_path}: 충분한 그룹(진동, 운영)이 없습니다. 스킵합니다.")
            return None, None

        vib_group = tdms_file.groups()[0]
        operation_group = tdms_file.groups()[1]

        vib_data_dict = {}
        for ch_raw in channels_raw:
            ch_stripped = ch_raw.strip()
            vib_channel_names = [ch.name.strip() for ch in vib_group.channels()]
            if ch_stripped in vib_channel_names:
                matched_channel = next(ch for ch in vib_group.channels() if ch.name.strip() == ch_stripped)
                vib_data_dict[ch_stripped] = matched_channel.data
            else:
                vib_data_dict[ch_stripped] = np.array([])

        operation_data_dict = {}
        for ch_raw in other_channels_raw:
            ch_stripped = ch_raw.strip()
            operation_channel_names = [ch.name.strip() for ch in operation_group.channels()]
            if ch_stripped in operation_channel_names:
                matched_channel = next(ch for ch in operation_group.channels() if ch.name.strip() == ch_stripped)
                operation_data_dict[ch_stripped] = matched_channel.data
            else:
                operation_data_dict[ch_stripped] = np.array([])
        return vib_data_dict, operation_data_dict

    except Exception as e:
        print(f"[에러] {file_path} 로드 중 오류 발생: {e}")
        return None, None


# --- EOL 인덱스 찾기 ---
def _find_eol_index(tdms_filenames, folder_path, TORQUE_THRESHOLD, TEMP_THRESHOLD, channels_raw, other_channels_raw):
    eol_file_idx = -1
    for i, file_name in enumerate(tdms_filenames):
        file_path = os.path.join(folder_path, file_name)
        # _load_tdms_data는 이제 vib_data_dict와 operation_data_dict를 반환
        _, operation_data_dict = _load_tdms_data(file_path, channels_raw, other_channels_raw)

        if operation_data_dict is None: # 파일 로드 실패 또는 그룹 부족 시
            continue

        # if operation_data_dict is not None:
        #     print(f"DEBUG: [{file_name}] operation_data_dict keys: {list(operation_data_dict.keys())}")
        # else:
        #     print(f"DEBUG: [{file_name}] operation_data_dict is None")
        # print(f"DEBUG: [{file_name}] operation_data_dict keys: {[repr(k) for k in operation_data_dict.keys()]}")


        torque_data = operation_data_dict.get('Torque[Nm]'.strip(), np.array([0]))
        temp_front_data = operation_data_dict.get('TC SP Front[℃]'.strip(), np.array([0]))
        temp_rear_data = operation_data_dict.get('TC SP Rear[℃]'.strip(), np.array([0]))

        # EOL 조건 확인 (데이터가 비어있지 않은 경우에만 min/max 계산)
        torque_cond = (torque_data.size > 0) and (np.min(torque_data) <= TORQUE_THRESHOLD)
        temp_front_cond = (temp_front_data.size > 0) and (np.max(temp_front_data) >= TEMP_THRESHOLD)
        temp_rear_cond = (temp_rear_data.size > 0) and (np.max(temp_rear_data) >= TEMP_THRESHOLD)

        if torque_cond or temp_front_cond or temp_rear_cond:
            eol_file_idx = i
            break

    if eol_file_idx == -1:
        print(f"[경고] {folder_path}: EOL 조건이 만족되지 않았습니다. 마지막 파일을 EOL로 간주합니다.")
        eol_file_idx = len(tdms_filenames) - 1
        if eol_file_idx < 0: # 폴더에 파일이 아예 없는 경우
            return -1 # 유효한 파일 없음

    return eol_file_idx



# --- 단일 윈도우 특징 추출 ---
def _extract_features_from_window(vib_window, operation_data_dict, channels_raw, other_channels_raw, wavelet, level, top_k):
    # 하나의 슬라이딩 윈도우와 해당 TMDS 파일의 operation data에서 특징을 추출하고 결합한다.
    per_channel_vib_features = []
    for ch_idx in range(vib_window.shape[1]):
        vib_feat = extract_wpt_fft_features(vib_window[:, ch_idx], wavelet, level, top_k)
        per_channel_vib_features.append(vib_feat)

    current_other_features = []
    for ch_other in other_channels_raw:
        # 운영 데이터는 TDMS 파일당 하나이므로, 윈도우별로 반복해서 추가
        data = operation_data_dict.get(ch_other, np.array([]))
        if data.size > 0:
            current_other_features.extend([np.mean(data), np.std(data), np.sqrt(np.mean(data**2))])
        else:
            current_other_features.extend([0.0, 0.0, 0.0]) # 데이터 없을 시 0으로 채움

    combined_features = np.concatenate(per_channel_vib_features + [np.array(current_other_features)])
    return combined_features

# --- 특징 추출 및 RUL 추가 ---
def extract_features_and_rul(folder_path, channels_raw, other_channels_raw, window_size, overlap, sampling_rate, wavelet, level, top_k):
    # 중단 조건 값은 여기서는 인자로 받지 않음 (EOL 탐색 함수 내부에서 사용)
    TORQUE_THRESHOLD = -17.0
    TEMP_THRESHOLD = 200.0

    tdms_filenames = sorted([f for f in os.listdir(folder_path) if f.endswith(".tdms")])
    if not tdms_filenames:
        print(f"[스킵] {folder_path}: TDMS 파일이 없습니다.")
        expected_feature_dim = top_k * (2 ** level) * len(channels_raw) + len(other_channels_raw) * 3
        return np.empty((0, expected_feature_dim)), np.empty((0,))

    # 1. EOL 파일 인덱스 찾기
    eol_file_idx = _find_eol_index(tdms_filenames, folder_path, TORQUE_THRESHOLD, TEMP_THRESHOLD, channels_raw, other_channels_raw)

    if eol_file_idx == -1:
        # EOL 파일을 찾지 못했고, 폴더에도 파일이 없거나 유효한 파일이 없는 경우
        expected_feature_dim = top_k * (2 ** level) * len(channels_raw) + len(other_channels_raw) * 3
        return np.empty((0, expected_feature_dim)), np.empty((0,))

    # EOL까지의 파일 목록
    tdms_filenames_until_eol = tdms_filenames[:eol_file_idx + 1]

    if not tdms_filenames_until_eol:
        print(f"[스킵] {folder_path}: EOL까지 유효한 TDMS 파일이 없습니다.")
        expected_feature_dim = top_k * (2 ** level) * len(channels_raw) + len(other_channels_raw) * 3
        return np.empty((0, expected_feature_dim)), np.empty((0,))

    all_window_features = []
    all_window_ruls = []

    # 2. EOL까지의 총 윈도우 개수를 정확히 계산 (RUL 라벨링을 위해 필요)
    actual_total_windows_until_eol = 0

    for i, file_name in enumerate(tdms_filenames_until_eol):
        file_path = os.path.join(folder_path, file_name)
        # _load_tdms_data는 이제 vib_data_dict와 operation_data_dict를 반환
        vib_data_dict, _ = _load_tdms_data(file_path, channels_raw, other_channels_raw)

        if vib_data_dict is None: # 파일 로드 실패 또는 그룹 부족 시
            continue

        # 진동 데이터만 사용하여 윈도우 수 계산
        min_len_file_vib = float('inf')
        valid_vib_channels_exist = False
        vib_data_for_stacking = [] # 유효한 진동 데이터만 담을 리스트

        for ch_raw in channels_raw:
            ch_stripped = ch_raw.strip()
            data = vib_data_dict.get(ch_stripped, np.array([]))
            if data.size > 0:
                min_len_file_vib = min(min_len_file_vib, data.size)
                valid_vib_channels_exist = True
                vib_data_for_stacking.append(data)
            else:
                vib_data_for_stacking.append(np.array([])) # 누락 채널은 빈 배열로 추가

        if not valid_vib_channels_exist or min_len_file_vib < window_size:
            continue

        # min_len_file_vib 길이로 통일하고 스택
        final_vib_data_to_stack = []
        for data_arr in vib_data_for_stacking:
            if data_arr.size > 0:
                final_vib_data_to_stack.append(data_arr[:min_len_file_vib])
            else: # 채널은 있지만 데이터가 아예 없었던 경우, min_len_file_vib 길이로 0 채움
                final_vib_data_to_stack.append(np.zeros(min_len_file_vib))

        dummy_vib_data_stacked = np.stack(final_vib_data_to_stack, axis=-1)
        dummy_vib_windows = sliding_window(dummy_vib_data_stacked, window_size, overlap)
        actual_total_windows_until_eol += len(dummy_vib_windows)

    if actual_total_windows_until_eol == 0:
        print(f"[경고] {folder_path}: EOL까지 유효한 윈도우가 전혀 생성되지 않았습니다.")
        expected_feature_dim = top_k * (2 ** level) * len(channels_raw) + len(other_channels_raw) * 3
        return np.empty((0, expected_feature_dim)), np.empty((0,))


    # 3. 실제 특징 추출 및 RUL 라벨링
    current_overall_window_idx = 0 # 폴더 내의 전체 윈도우 인덱스 (0부터 시작)

    for i, file_name in enumerate(tdms_filenames_until_eol):
        file_path = os.path.join(folder_path, file_name)
        # _load_tdms_data는 이제 vib_data_dict와 operation_data_dict를 반환
        vib_data_dict, operation_data_dict = _load_tdms_data(file_path, channels_raw, other_channels_raw)

        if vib_data_dict is None or operation_data_dict is None:
            continue

        # 진동 데이터만 추출하여 윈도우 생성
        min_len_file_vib = float('inf')
        valid_vib_channels_exist = False
        vib_data_for_stacking = []
        for ch_raw in channels_raw:
            ch_stripped = ch_raw.strip()
            data = vib_data_dict.get(ch_stripped, np.array([]))
            if data.size > 0:
                min_len_file_vib = min(min_len_file_vib, data.size)
                valid_vib_channels_exist = True
                vib_data_for_stacking.append(data)
            else:
                vib_data_for_stacking.append(np.array([])) # 누락 채널은 빈 배열로 추가

        if not valid_vib_channels_exist or min_len_file_vib < window_size:
            continue

        # min_len_file_vib 길이로 통일하고 스택
        final_vib_data_to_stack = []
        for data_arr in vib_data_for_stacking:
            if data_arr.size > 0:
                final_vib_data_to_stack.append(data_arr[:min_len_file_vib])
            else: # 채널은 있지만 데이터가 아예 없었던 경우, min_len_file_vib 길이로 0 채움
                final_vib_data_to_stack.append(np.zeros(min_len_file_vib))

        current_file_vib_stacked = np.stack(final_vib_data_to_stack, axis=-1)
        vib_windows = sliding_window(current_file_vib_stacked, window_size, overlap)

        if len(vib_windows) == 0:
            print(f"[경고] {file_name}: 슬라이딩 윈도우 결과 없음 (진동 데이터 기준). 스킵합니다.")
            continue

        for w_idx in range(len(vib_windows)):
            current_vib_window = vib_windows[w_idx]

            # 특징 추출 보조 함수 호출
            combined_features = _extract_features_from_window(
                current_vib_window, operation_data_dict, channels_raw, other_channels_raw, wavelet, level, top_k
            )
            all_window_features.append(combined_features)

            # RUL 라벨링: EOL까지 남은 총 윈도우 개수를 기준으로 선형 감소
            rul_for_this_window = (actual_total_windows_until_eol - 1 - current_overall_window_idx) * (step / sampling_rate)
            rul_for_this_window = max(0, rul_for_this_window) # 음수가 되지 않도록
            all_window_ruls.append(rul_for_this_window)

            current_overall_window_idx += 1

    if not all_window_features:
        expected_feature_dim = top_k * (2 ** level) * len(channels_raw) + len(other_channels_raw) * 3
        return np.empty((0, expected_feature_dim)), np.empty((0,))

    feature_matrix = np.array(all_window_features)
    rul_seconds = np.array(all_window_ruls)

    return feature_matrix, rul_seconds

평가 파이프라인

In [6]:
# 오차 계산
def calculate_error(ActRUL, PredRUL):
    # error = 100 * (ActRUL - PredRUL) / ActRUL
    # return error
    # ActRUL이 0이거나 매우 작은 값일 경우를 대비하여 epsilon을 더합니다.
    # 이는 RUL이 0인 경우를 '수명이 완전히 다함'으로 간주하고,
    # 해당 시점의 예측 오차를 패널티하는 방식입니다.
    # 그러나 이것은 엄밀히 ActRUL=0일 때의 정의를 변경하는 것이므로 주의가 필요합니다.

    # ActRUL이 0일 때 (진정한 수명 종료 시점) PredRUL도 0이면 오차를 0으로.
    # ActRUL이 0인데 PredRUL이 0이 아니면 예측이 틀린 것이므로, 큰 패널티를 부여.
    # 여기서는 기존 평가 함수에 맞추기 위해 100 * (ActRUL - PredRUL) / ActRUL 형태를 유지하되,
    # ActRUL이 0인 경우 (나눗셈 불가능)를 별도로 처리합니다.

    # Numpy 배열 연산을 위해 np.where 사용
    # ActRUL이 0인 경우 (매우 민감한 지점)
    #   - PredRUL도 0이면 오차 0 (정확히 예측)
    #   - PredRUL이 0이 아니면 (수명이 다했는데도 남았다고 예측) -> 큰 음의 오차 (패널티)
    # ActRUL이 0이 아닌 경우 (일반적인 RUL 예측)
    #   - 정상적인 오차 계산
    error = np.where(ActRUL == 0,
                     np.where(PredRUL == 0, 0, 100 * (-PredRUL) / 1), # PredRUL이 양수면 음의 오차 (큰 패널티), PredRUL이 음수일 일은 없다고 가정
                     100 * (ActRUL - PredRUL) / ActRUL)
    return error

# 정확도 점수 계산
def calculate_accuracy_score(error):
    ln_0_5 = np.log(0.5)
    score = np.where(
        error <= 0,
        np.exp(-ln_0_5 * error / 20),
        np.exp(+ln_0_5 * error / 20)
    )
    return score

# 최종 점수 계산
def calculate_final_score(accuracy_scores):
    return np.mean(accuracy_scores)

# 전체 평가 파이프라인
def evaluate_rul_prediction(ActRUL, PredRUL):
    error = calculate_error(ActRUL, PredRUL)
    accuracy_scores = calculate_accuracy_score(error)
    final_score = calculate_final_score(accuracy_scores)
    return {
        "Error": error,
        "Accuracy_scores": accuracy_scores,
        "Final_Score": final_score
    }

모델 로드 및 예측 수행 후 평가 점수 출력

In [7]:
# 저장된 모델 로드
model_path = "/content/drive/MyDrive/KSPHM-data-challenge/model/cnn_lstm_model_2.h5"
model = load_model(model_path, compile=False)
print("모델 로드 완료:", model_path)

# 저장된 스케일러 로드
scaler_dir = "/content/drive/MyDrive/KSPHM-data-challenge/scalers"
X_scaler = joblib.load(os.path.join(scaler_dir, 'X_scaler.pkl'))
y_scaler = joblib.load(os.path.join(scaler_dir, 'y_scaler.pkl'))
print("스케일러 로드 완료")

# Validation 폴더 경로
validation_base_path = "/content/drive/MyDrive/KSPHM-data-challenge/Validation Set"
validation_folders = [f"Validation{i}" for i in range(1, 7)]

results = []

for folder_name in validation_folders:
    folder_path = os.path.join(validation_base_path, folder_name)
    print(f"\n--- {folder_name} 데이터 처리 중 ---")

    # 1. Validation 데이터 로드 및 특징 추출
    features, actual_rul = extract_features_and_rul(
        folder_path, channels_raw, other_channels_raw,
        window_size, overlap, sampling_rate, wavelet, level, top_k
    )

    if features.shape[0] == 0:
        print(f"Skipping {folder_name} due to no valid features extracted.")
        results.append({
            "File": folder_name,
            "RUL_Score": np.nan
        })
        continue

    # 1-1. 입력 데이터 스케일링 적용
    features_scaled = X_scaler.transform(features)

    n_samples = features_scaled.shape[0]
    n_features_per_window = features_scaled.shape[1]

    # 모델 입력 형태로 reshape
    reshaped_features = features_scaled.reshape(n_samples, n_features_per_window, 1)

    # 2. 저장된 모델로 RUL 예측 수행
    try:
        predicted_rul_scaled = model.predict(reshaped_features).flatten()

        # 2-1. 스케일링 역변환하여 원래 RUL 값으로 복원
        predicted_rul = y_scaler.inverse_transform(predicted_rul_scaled.reshape(-1,1)).flatten()

        # 음수 예측값 방지
        predicted_rul[predicted_rul < 0] = 0

        # 실제 RUL과 예측 RUL 길이 맞추기
        min_len = min(len(actual_rul), len(predicted_rul))
        actual_rul_trimmed = actual_rul[:min_len]
        predicted_rul_trimmed = predicted_rul[:min_len]

        # 3. 평가 점수 계산
        evaluation_results = evaluate_rul_prediction(actual_rul_trimmed, predicted_rul_trimmed)
        final_score = evaluation_results["Final_Score"]
        print(f"{folder_name} Final Score: {final_score:.4f}")

        results.append({
            "File": folder_name,
            "RUL_Score": final_score
        })
    except Exception as e:
        print(f"[에러] {folder_name} 예측 중 오류 발생: {e}")
        results.append({
            "File": folder_name,
            "RUL_Score": np.nan
        })

# 결과 저장
df_results = pd.DataFrame(results)
team_name = "파머완"
output_excel_path = f"/content/drive/MyDrive/KSPHM-data-challenge/RUL_Score/{team_name}_validation3.xlsx"
df_results.to_excel(output_excel_path, index=False)
print(f"\n평가 점수 저장 완료: {output_excel_path}")

모델 로드 완료: /content/drive/MyDrive/KSPHM-data-challenge/model/cnn_lstm_model_2.h5


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


스케일러 로드 완료

--- Validation1 데이터 처리 중 ---
[경고] /content/drive/MyDrive/KSPHM-data-challenge/Validation Set/Validation1: EOL 조건이 만족되지 않았습니다. 마지막 파일을 EOL로 간주합니다.
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Validation1 Final Score: 0.2117

--- Validation2 데이터 처리 중 ---


<ipython-input-6-a1f03f38282e>:23: RuntimeWarning: divide by zero encountered in divide
  100 * (ActRUL - PredRUL) / ActRUL)
<ipython-input-6-a1f03f38282e>:32: RuntimeWarning: overflow encountered in exp
  np.exp(+ln_0_5 * error / 20)


[경고] /content/drive/MyDrive/KSPHM-data-challenge/Validation Set/Validation2: EOL 조건이 만족되지 않았습니다. 마지막 파일을 EOL로 간주합니다.
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Validation2 Final Score: 0.0273

--- Validation3 데이터 처리 중 ---


...

[경고] /content/drive/MyDrive/KSPHM-data-challenge/Validation Set/Validation5: EOL 조건이 만족되지 않았습니다. 마지막 파일을 EOL로 간주합니다.
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Validation5 Final Score: 0.0236

--- Validation6 데이터 처리 중 ---


<ipython-input-6-a1f03f38282e>:23: RuntimeWarning: invalid value encountered in divide
  100 * (ActRUL - PredRUL) / ActRUL)


[경고] /content/drive/MyDrive/KSPHM-data-challenge/Validation Set/Validation6: EOL 조건이 만족되지 않았습니다. 마지막 파일을 EOL로 간주합니다.
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Validation6 Final Score: 0.0000


<ipython-input-6-a1f03f38282e>:23: RuntimeWarning: divide by zero encountered in divide
  100 * (ActRUL - PredRUL) / ActRUL)
<ipython-input-6-a1f03f38282e>:32: RuntimeWarning: overflow encountered in exp
  np.exp(+ln_0_5 * error / 20)



평가 점수 저장 완료: /content/drive/MyDrive/KSPHM-data-challenge/RUL_Score/파머완_validation3.xlsx
